# Transmissão entre Antenas

![Experimento](transmissao/experimento.jpg)

Nesse experimento verificamos a formação de uma onda estacionária entre antenas transmissora e receptora.

A partir da distância entre picos (ou vales) é possível determinarmos a frequência do sinal transmitido.

In [ ]:
# Import necessary modules
import numpy
from matplotlib import pyplot

# Only in Jupyter Notebooks
%matplotlib notebook

Os dados estão armazenados em um arquivo CSV (*comma-separated values*).

Usamos o comando `%cat` (Jupyter Notebook ou terminal IPython) para mostrar o conteúdo de arquivos locais.

In [ ]:
%cat transmissao/dados.csv

É possível ler arquivos em python diretamente, mas teríamos que interpretar e converter os dados manualmente.

```python
data = []
with open('transmissao/dados.csv') as infile:
    print(infile.readline())
    for line in infile:
        data.append([float(i) for i in line.split(';')])
data = numpy.array(data)
```

Contudo o módulo numpy oferece a função `loadtxt` para esse fim.
Além de mais simples, ela é também mais rápida.

In [ ]:
numpy.loadtxt?

In [ ]:
data = numpy.loadtxt('transmissao/dados.csv', delimiter=';', skiprows=1)
data

In [ ]:
data.shape

In [ ]:
data[0, :]

In [ ]:
data[0, :].shape

Separamos os valores de distância da 1ª coluna no vetor `x` e as medidas em `v`:

In [ ]:
x = data[:, 0]
x

In [ ]:
v = data[:, 1:]
v

In [ ]:
fig, ax = pyplot.subplots(1, 1)

ax.plot(x, v[:, 0], '.')
ax.plot(x, v[:, 1], '.')
ax.plot(x, v[:, 2], '.')

Podemos analisar os dados em termos de médias e desvios padrão:

In [ ]:
v0 = v[0, :]
print(v0, numpy.mean(v0), numpy.std(v0))

In [ ]:
%%timeit
y = numpy.zeros(len(x))
for i in range(len(x)):
    y[i] = numpy.mean(v[i, :])

In [ ]:
%%timeit
y = numpy.mean(v, axis=1)

In [ ]:
y = numpy.mean(v, axis=1)
e = numpy.std(v, axis=1)
e.shape

In [ ]:
fig, ax = pyplot.subplots(1, 1)

ax.errorbar(x, y, e, capsize=3)

ax.set_xlabel('Distance (mm)')
ax.set_ylabel('Signal (normalized)')

ax.grid()

Há mais de uma maneira de buscar a distância entre máximos.
Simplesmente olhar para o gráfico é uma delas, mas imagine que temos centenas ou milhares desses conjuntos de dados.
Nesse caso é melhor buscarmos um método mais automático de análise.

Devido ao ruído, usar simplesmente `numpy.max` pode resultar em baixa precisão.

Outra opção é usar algum tipo de filtragem (o módulo `scipy.signal` possui diversas opções de filtros), mas também funcionaria melhor se tivéssemos maior densidade de pontos.

Para os dados disponíveis, podemos usar o fato de que o sinal é quase senoidal e usar a transformada discreta de Fourier para determinar o coeficiente de maior magnitude (excluindo a frequência 0).

In [ ]:
c = numpy.abs(numpy.fft.rfft(y))
k = numpy.fft.rfftfreq(len(y), x[1] - x[0])

fig, ax = pyplot.subplots(1, 1)

ax.semilogy(k, c, 'o-')

ax.set_xlabel('Spacial frequency (mm⁻¹)')
ax.set_ylabel('Fourier coeff. magnitude')

In [ ]:
numpy.argmax?

In [ ]:
i = numpy.argmax(c[1:]) + 1
i

In [ ]:
ax.annotate?

In [ ]:
fig, ax = pyplot.subplots(1, 1)

ax.semilogy(k, c, '.-')
ax.annotate(xy=(k[i], c[i]), s='Max.',
            xytext=(k[i] + 0.05, c[i] * 0.7),
            arrowprops={'arrowstyle': '->'})

ax.set_xlabel('Spacial frequency (mm⁻¹)')
ax.set_ylabel('Fourier coeff. magnitude')

lda = 2 / k
f = 3e8 / (lda * 1e-3)
print('Frequency: ({:.1f} ± {:.1f}) GHz'.format(f[i]*1e-9, (f[i+1] - f[i-1]) / 2e9))

Aqui também a esparcidade de dados limita a precisão.

A solução neste caso é usar nosso conhecimento do experimento: sabemos que uma onda estacionária tem distribuição espacial de potência senoidal.
Portanto, podemos ajustar um modelo aos dados e assim determinar a frequência da melhor senoide que reproduz os dados coletados (similarmente ao que fizemos com o ajuste linear usando `numpy.polyfit`).

In [ ]:
from scipy.optimize import curve_fit

curve_fit?

Nosso modelo deve ser uma função da posição e dos parâmetros que queremos otimizar.
Por se tratar de um problema não-linear, é preciso dar um chute inicial razoável para o algoritmo de ajuste.

In [ ]:
def model(x, k, a, x0, y0):
    return y0 + a * numpy.sin(2*numpy.pi*k*(x - x0))

model(0.25, 1, 1, 0, 0)

In [ ]:
model(numpy.array([0, 0.25, 0.5, 0.75, 1]), 1, 1, 0, 0)

In [ ]:
init = (0.08, 0.2, 291, 0.75)

fit_x = numpy.linspace(x[0], x[-1], 201)

init_y = model(fit_x, *init)

fig, ax = pyplot.subplots(1, 1)

ax.errorbar(x, y, e, capsize=3, ls='', marker='.')
ax.plot(fit_x, init_y)

In [ ]:
opt, cov = curve_fit(model, x, y, init)

opt, numpy.sqrt(numpy.diag(cov))

In [ ]:
fit_y = model(fit_x, *opt)

fig, ax = pyplot.subplots(1, 1)

ax.errorbar(x, y, e, capsize=3, ls='', marker='.')
ax.plot(fit_x, fit_y)

ax.set_xlabel('Distance (mm)')
ax.set_ylabel('Signal (normalized)')

ax.grid()

Conhecendo o modelo, sabemos qua a distância entre máximos é a metade do comprimento de onda do sinal transmitido.

In [ ]:
lda = 2 / opt[0]
f = 3e8 / (lda * 1e-3)
print('Frequency: {:.1f} GHz'.format(f*1e-9))

## Exercício

Modifique o modelo para usar diretamente a frequência do sinal, possibilitando a determinação da confiabilidade desse parâmetro diretamente da matriz de covariância.

Podemos usar um modelo mais físico para resolver o problema:

In [ ]:
def model2(x, f, a, e0, g, p0, p1):
    phi = (a + 6.67e-3j * numpy.pi * f) * (x - 280)
    y = e0 * (numpy.exp(1j * p0 - phi) + g * numpy.exp(1j * p1 + phi))
    return numpy.abs(y)**2

init2 = (10, 0.001, 0.9, 0.1, 0, 0)
opt2, cov2 = curve_fit(model2, x, y, init2)

init_y2 = model2(fit_x, *init2)
fit_y2 = model2(fit_x, *opt2)

fig, ax = pyplot.subplots(2, 1, figsize=(6, 6))

for a in ax:
    a.errorbar(x, y, e, capsize=3, ls='', marker='.')
    a.set_ylabel('Signal (normalized)')
    a.grid()
ax[1].set_xlabel('Distance (mm)')

ax[0].plot(fit_x, init_y2)
ax[1].plot(fit_x, fit_y2)

print('Frequency: ({:.2f} ± {:.2f}) GHz'.format(opt2[0], cov2[0, 0]**0.5))